# EDCSA from Scratch

## Homework Guidelines

Implement ECDSA from scratch. You want to use the secp256k1 curve. See here for a reference: https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python

1) pick a private key

2) generate the public key using that private key (not the eth address, the public key)

3) pick message m and hash it to produce h (h can be though of as a 256 bit number)

4) sign m using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)

5) verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch

## Answer

First we will set up an object to define the parameters of the secp256k1 curve:

p (int): The value of `p` in the curve equation.
a (int): The value of `a` in the curve equation.
b (int): The value of `b` in the curve equation.
q (int): The order of the base point of the curve.
gx (int): The x coordinate of the base point of the curve.
gy (int): The y coordinate of the base point of the curve.

(Also some helper functions to assist with checking is the point if on the curve etc)

In [175]:
class Secp256k1:
    def __init__(self):
        self.p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
        self.a = 0x0
        self.b = 0x7
        self.n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
        self.gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
        self.gy = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
    
    def is_given_point_on_the_curve(self, point: (int, int)) -> bool:
        x, y, = point
        if x == 0 and y == 0:
            return True # Identity should be on the curve
        left = (y**2) % self.p
        right = (x**3 + self.a * x + self.b) % self.p
        return left == right
    
    def check(self, x: int) -> int:
        return (x ** 3 + self.a * x + self.b) % self.p
    
    def get_G(self) -> (int,int):
        return (self.gx,self.gy)
    
    def __str__(self):
        return f"P|{self.p}|A={self.a}|B={self.b}|N={self.n}|Gx={self.gx}|Gy={self.gy}"


Next define a class where we can encompass the point arithmetic

In [177]:
class Point:
    def __init__(self, x: int, y: int, curve: Secp256k1):
        if curve is not None:
            x = x % curve.p
            y = y % curve.p

        if not (x == 0 and y == 0) and not curve.is_given_point_on_the_curve((x, y)):
            raise ValueError(
                'x and y are not on curve x={} y={}'.format(x, y))
        else:
            self.x = x
            self.y = y
            self.curve = curve
            
    def __str__(self):
        return f"X={self.x}|Y={self.y} \nCurve={self.curve}"


class Point_Utils:
    
    def double(that:Point) -> Point:
        lambda_ = (3 * that.x**2 + that.curve.a) * pow(2 * that.y, -1, that.curve.p)
        x_r = (lambda_**2 - 2 * that.x) % that.curve.p
        y_r = (lambda_ * (that.x - x_r) - that.y) % that.curve.p
        return Point(x_r, y_r,that.curve)
                
    def equals(this:Point,that:Point) -> bool:
        return this.x == that.x and this.y == that.y 
    
    def not_equal(this:Point, that:Point) -> bool:
        return not Point_Utils.equals(this,that)
    
    def add(this:Point, that:Point) -> Point:
        if this is None or this.curve is None:
            return that
       
        if that is None or that.curve is None:
            return this
        
        if Point_Utils.equals(this,that):
            return Point_Utils.double(that)
                          
        if this.x == that.x:
            return Point.IDENTITIY_ELEMENT
        
        lambda_ = (that.y - this.y) * pow(that.x - this.x, -1,this.curve.p)

        x3 = (lambda_**2 - this.x - that.x) % this.curve.p
        y3 = (lambda_ * (this.x - x3) - this.y) % this.curve.p

        return Point(x3, y3, this.curve)
        
        
        
    def sub(this:Point, that:Point) -> Point:
        return add(this, Point(-that.x, -that.y))
    
    def mul(that:Point, k:int) -> Point:
        R0 = Point.IDENTITY_ELEMENT
        R1 = that
        for bit in bin(k)[2:]:
            if bit == '0':
                R1 = Point_Utils.add(R0,R1)
                R0 = Point_Utils.double(R0)
            else:
                R0 = Point_Utils.add(R0, R1)
                R1 = Point_Utils.double(R1)
        return R0
        
    Point.IDENTITY_ELEMENT = Point(0, 0, None) 
   

Test to assert that double and additon result in points that are on the curve

In [178]:
curve = Secp256k1()
g_x,g_y = curve.get_G()

a = Point(g_x,g_y,curve )
print("Curve a is {}".format(a))

b = Point_Utils.double(a)
assert Point_Utils.not_equal(a,b)
print("Curve b is {}".format(b))

c = Point_Utils.add(a,b)
assert Point_Utils.not_equal(a,c)
assert Point_Utils.not_equal(c,b)
print("Curve c is {}".format(c))

assert c.curve.is_given_point_on_the_curve((c.x,c.y)) == True


Curve a is X=55066263022277343669578718895168534326250603453777594175500187360389116729240|Y=32670510020758816978083085130507043184471273380659243275938904335757337482424 
Curve=P|115792089237316195423570985008687907853269984665640564039457584007908834671663|A=0|B=7|N=115792089237316195423570985008687907852837564279074904382605163141518161494337|Gx=55066263022277343669578718895168534326250603453777594175500187360389116729240|Gy=32670510020758816978083085130507043184471273380659243275938904335757337482424
Curve b is X=89565891926547004231252920425935692360644145829622209833684329913297188986597|Y=12158399299693830322967808612713398636155367887041628176798871954788371653930 
Curve=P|115792089237316195423570985008687907853269984665640564039457584007908834671663|A=0|B=7|N=115792089237316195423570985008687907852837564279074904382605163141518161494337|Gx=55066263022277343669578718895168534326250603453777594175500187360389116729240|Gy=3267051002075881697808308513050704318447127338065924327593

Test to assert that scalar multiplcation result in a point that is on the curve

In [179]:
k = 11
x = Point(g_x,g_y,curve )
y = Point_Utils.mul(x,k)


assert Point_Utils.not_equal(x,y)
print("Curve y is {}".format(y))

assert y.curve.is_given_point_on_the_curve((y.x,y.y))


Curve y is X=53957576663012291606402345341061437133522758407718089353314528343643821967563|Y=98386217607324929854432842186271083758341411730506808463586570492533445740059 
Curve=P|115792089237316195423570985008687907853269984665640564039457584007908834671663|A=0|B=7|N=115792089237316195423570985008687907852837564279074904382605163141518161494337|Gx=55066263022277343669578718895168534326250603453777594175500187360389116729240|Gy=32670510020758816978083085130507043184471273380659243275938904335757337482424


We create a class to generate the pub-prive key pair

In [183]:
import random

class Keys:
    def _generate_priv_key(self,curve:Secp256k1) -> int:
        return random.randint(1, curve.n - 1)
    
    def _generate_pub_key(self,curve:Secp256k1, private_key:int):
        public_key = None
        gx,gy = curve.get_G()
        Q = Point(gx,gy, curve)
        print("Q is {}".format(Q))
        for _ in range(2):
            Q = Point_Utils.double(Q)
            
        print("Q is now {}".format(Q))
        public_key = None
        for bit in reversed(bin(private_key)[2:]):
            if public_key is None:
                public_key = Q if bit == '1' else None
            else:
                public_key = Point_Utils.add(public_key, public_key)
        
            if bit == '1':
                public_key = Point_Utils.add(public_key, Q)
                
        return public_key
       
        
    def __init__(self, curve:Secp256k1):
            priv = self._generate_priv_key(curve)
            pub = self._generate_pub_key(curve, priv)
            self.priv = priv
            self.pub = pub
            self.priv_hash = hex(priv)[2:]
            self.pub_hash = f"04{pub.x:x}{pub.y:x}"
            print("Key is {}".format(self))
    
    def __str__(self):
        return f"PRIV={self.priv} | PUB={self.pub}"

          

Test the SHA256 with an expected hash

In [184]:
import hashlib
print("Version is {}",sys.version_info.major)
message = b"Yay it works!"
hashed_message = hashlib.sha256(message).hexdigest()
print("Hashed message is {}",hashed_message)
expected = "ff90787c2e04b59bbc23647a4d00026cafd4b24e6538a0cc60cf20fe992c1c26"
assert expected == hashed_message

Version is {} 3
Hashed message is {} ff90787c2e04b59bbc23647a4d00026cafd4b24e6538a0cc60cf20fe992c1c26


In [201]:
import os
import hashlib

class EDCSA:
    def __init__(self, curve:Secp256k1, nonce:int):
        gx,gy = curve.get_G()
        self.name = "EDCSA"
        self.k = nonce
        self.G = Point(gx,gy,curve)
        self.R = Point_Utils.mul(self.G,nonce)
    
    
    def sign(self,message:str, curve:Secp256k1, private_key:int):
        encoded = message.encode('UTF-8')
        e = int(hashlib.sha256(encoded).hexdigest(),16)
        r = 0
        s = 0
        while r == 0 or s == 0:
            r = self.R.x % curve.n
            inv = pow(k,-1,curve.n)
            rKey = private_key*r
            print("inv is {}".format(inv))
            print("rKey is {}".format(rKey))
            s = (inv * (e + rKey)) % curve.n
        return (r,s)
            
        
    def verify(self,message:str, curve, signature, public_key):
        encoded = message.encode('UTF-8')
        e = int(hashlib.sha256(encoded).hexdigest(),16)
        r,s = signature
        w = pow(s,-1,curve.n)
        u1 = (e * w) % curve.n
        u2 = (r * w) % curve.n
        P_1 = Point_Utils.mul(self.G,u1)
        P_2 = Point_Utils.mul(public_key,u2)
        P = Point_Utils.add(P_1,P_2)   
        print("P.x is {}".format( P.x))
        print("P.y is {}".format(P.y))
        print("r is {}".format( r))
        print("q is {}".format(curve.n))
        print("P.x % curve.q is {}".format(P.x % curve.n))
        
        return (P.x == r)
        

Sign and verify a message using EDCSA

In [202]:

keys = Keys(curve)
priv = keys.priv
pub = keys.pub
msg = "Yay it works!"
k=500
edcsa = EDCSA(curve,k)

signature = edcsa.sign(message = msg, curve = curve, private_key = priv)
print("Signed message is {}".format(signed)) 
is_valid = edcsa.verify(msg,curve, signature, pub)
if is_valid:
    print("Message is valid")
else:
    print("Message has been tampered with")


Q is X=55066263022277343669578718895168534326250603453777594175500187360389116729240|Y=32670510020758816978083085130507043184471273380659243275938904335757337482424 
Curve=P|115792089237316195423570985008687907853269984665640564039457584007908834671663|A=0|B=7|N=115792089237316195423570985008687907852837564279074904382605163141518161494337|Gx=55066263022277343669578718895168534326250603453777594175500187360389116729240|Gy=32670510020758816978083085130507043184471273380659243275938904335757337482424
Q is now X=103388573995635080359749164254216598308788835304023601477803095234286494993683|Y=37057141145242123013015316630864329550140216928701153669873286428255828810018 
Curve=P|115792089237316195423570985008687907853269984665640564039457584007908834671663|A=0|B=7|N=115792089237316195423570985008687907852837564279074904382605163141518161494337|Gx=55066263022277343669578718895168534326250603453777594175500187360389116729240|Gy=32670510020758816978083085130507043184471273380659243275938904335